## MAF Agent와 Azure AI Search 연동 예제

이 실습에서는 AI Search와 연결되어 있는 Agent를 생성하고, 해당 Agent를 이용하여 질의응답을 수행하는 예제입니다.

우선, AI Search를 사용하여 멀티모달로 지식 기반을 구성하는 방법은 다음 워크샵을 참고하세요

https://github.com/taeyo-kim/azure-ai-search-with-doc-intelligence

Agent의 instructions은 다음과 같이 입력합니다.

>```text
>당신은 연결된 Knowledge Base를 기반으로 하여 사용자에게 도움을 주는 에이전트입니다. Tool, 도구로 연결된 AI Search의 index를 기반으로  답변합니다. index에서 검색되지 않은 데이터에 대해서는 "제가 가진 지식 기반으로는 대답하기 어렵습니다" 라고 말해야 합니다. Tool을 사용하지 않는 대답은 모두 거절합니다.
>```

![Agent-AISearch1.png](./images/Agent-AISearch1.png)

In [ ]:
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import AgentReference
from azure.identity import DefaultAzureCredential

# 새로 생성할 에이전트 이름
AGENT_NAME = "DemoAgent"

AZURE_AI_PROJECT_ENDPOINT = "https://ms-foundry-lab-2058.services.ai.azure.com/api/projects/proj-default"
async with (
    AIProjectClient(
        endpoint=AZURE_AI_PROJECT_ENDPOINT,
        credential=DefaultAzureCredential()
    ) as project_client,
):
    # 기존 Agent 찾기
    agent = None
    async for existing in project_client.agents.list():
        if existing.name == AGENT_NAME:
            agent = existing
            break
    
    ## 기존 에이전트 가져오기
    if(agent):
        print(f"에이전트 '{AGENT_NAME}'(이)를 올바로 가져왔습니다. ")
    else:
        print(f"에이전트 '{AGENT_NAME}'(이)가 존재하지 않습니다")
        exit()      

    # OpenAI client 가져오기
    openai_client = project_client.get_openai_client()
    
    # Agent 정보를 본문에 추가하여 Response 생성 요청
    # AI Search에 KB에 있는 질문이므로 Agent가 대답할 수 있다.
    response = await openai_client.responses.create(
        input=[{"role": "user", "content": "sustainability?"}],
        extra_body={"agent": AgentReference(name=agent.name).as_dict()}
    )
    
    print(f"Agent response: {response.output_text}")

    # Agent 정보를 본문에 추가하여 Response 생성 요청
    # Agent는 이 질문에는 대답을 할 수 없다.
    response = await openai_client.responses.create(
        input=[{"role": "user", "content": "겨울에 어울리는 옷은 무엇인가요?"}],
        extra_body={"agent": AgentReference(name=agent.name).as_dict()}
    )
    
    print(f"Agent response: {response.output_text}")